In [7]:
import pandas as pd 

In [8]:
taxi_df = pd.read_parquet('Data/yellow_tripdata_2025-03.parquet')

In [9]:
taxi_df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee
0,1,2025-03-01 00:17:16,2025-03-01 00:25:52,1.0,0.90,1.0,N,140,236,1,7.9,3.50,0.5,2.60,0.0,1.0,15.50,2.5,0.0,0.00
1,1,2025-03-01 00:37:38,2025-03-01 00:43:51,1.0,0.60,1.0,N,140,262,1,6.5,3.50,0.5,2.30,0.0,1.0,13.80,2.5,0.0,0.00
2,2,2025-03-01 00:24:35,2025-03-01 00:39:49,1.0,1.94,1.0,N,161,68,1,14.9,1.00,0.5,5.16,0.0,1.0,25.81,2.5,0.0,0.75
3,2,2025-03-01 00:56:16,2025-03-01 01:01:35,2.0,0.95,1.0,N,231,13,1,7.2,1.00,0.5,2.59,0.0,1.0,15.54,2.5,0.0,0.75
4,1,2025-03-01 00:01:44,2025-03-01 00:10:00,1.0,1.50,1.0,N,163,236,1,8.6,4.25,0.5,2.85,0.0,1.0,17.20,2.5,0.0,0.75


In [10]:
taxi_df.shape

(4145257, 20)

In [11]:
taxi_df['pickup_dt']  = pd.to_datetime(taxi_df['tpep_pickup_datetime'])
taxi_df['dropoff_dt'] = pd.to_datetime(taxi_df['tpep_dropoff_datetime'])

taxi_df['duration_min'] = (taxi_df['dropoff_dt'] - taxi_df['pickup_dt']).dt.total_seconds() / 60

In [14]:
taxi_df[['duration_min']].head()

,duration_min
0,8.600000
1,6.216667
2,15.233333
3,5.316667
4,8.266667


In [15]:
taxi_df['duration_hr'] = taxi_df['duration_min'] / 60

In [16]:
taxi_df[['duration_hr']].head()

,duration_hr
0,0.143333
1,0.103611
2,0.253889
3,0.088611
4,0.137778


In [17]:
taxi_df['speed_mph'] = taxi_df['trip_distance'] / taxi_df['duration_hr']

In [18]:
taxi_df[['speed_mph']].head()

,speed_mph
0,6.279070
1,5.790885
2,7.641138
3,10.721003
4,10.887097


In [19]:
low_speed = taxi_df[(taxi_df['speed_mph'] < 3) & (taxi_df['trip_distance'] > 0.5)]

In [20]:
high_speed = taxi_df[taxi_df['speed_mph'] > 60]

In [21]:
low_speed.shape, high_speed.shape

((23961, 25), (22693, 25))

In [22]:
high_speed['speed_mph'].min(), high_speed['speed_mph'].max()

(np.float64(60.204968944099384), np.float64(inf))

In [24]:
bins = [60, 80, 100, 200, 500, 10000]
labels = ['60-80', '80-100', '100-200', '200-500', '500+']

high_speed.loc[:, 'speed_group'] = pd.cut(high_speed['speed_mph'], bins=bins, labels=labels)

high_speed['speed_group'].value_counts(normalize=True) * 100

speed_group
500+       34.293680
100-200    20.631970
60-80      19.144981
200-500    17.657993
80-100      8.271375
Name: proportion, dtype: float64

In [25]:
(high_speed.shape[0] / taxi_df.shape[0]) * 100

0.5474449473217222

In [27]:
high_speed['store_and_fwd_flag'].value_counts(normalize=True) * 100

store_and_fwd_flag
N    99.838464
Y     0.161536
Name: proportion, dtype: float64

In [28]:
high_speed['VendorID'].value_counts(normalize=True) * 100

VendorID
7    94.227295
1     2.934826
2     2.406028
6     0.431851
Name: proportion, dtype: float64

In [29]:
taxi_df['VendorID'].value_counts(normalize=True) * 100

VendorID
2    79.344851
1    20.128885
7     0.518207
6     0.008057
Name: proportion, dtype: float64